### Test notebook for logistic regression fit

In [2]:
%who

Interactive namespace is empty.


In [5]:
from dask.distributed import Client,Scheduler
from dask_jobqueue import SLURMCluster
cluster = SLURMCluster(cores=4,memory="31GB")
client = Client(cluster)
cluster.scale(cores=4)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.0.128.149:34533,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [1]:
! squeue -u ad9701

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
              8494      main dask-wor   ad9701  R      14:39      1 ood-n6
              8451       vdi sys/dash   ad9701  R    3:23:29      1 ood-vn15
              8493       vdi sys/dash   ad9701  R      19:58      1 ood-vn21


In [3]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 4
Total threads: 4,Total memory: 11.23 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33131,Workers: 4
Dashboard: /proxy/8787/status,Total threads: 4
Started: Just now,Total memory: 11.23 GiB
Comm: tcp://10.0.128.149:41153,Total threads: 1
Dashboard: /proxy/42715/status,Memory: 2.81 GiB
Nanny: tcp://127.0.0.1:33589,


In [5]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 4
Total threads: 4,Total memory: 11.23 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33131,Workers: 4
Dashboard: /proxy/8787/status,Total threads: 4
Started: 10 minutes ago,Total memory: 11.23 GiB
Comm: tcp://10.0.128.149:42619,Total threads: 1
Dashboard: /proxy/41751/status,Memory: 2.81 GiB
Nanny: tcp://127.0.0.1:33589,


In [3]:
%%time

## Fitting the model to P-E-Q by season

import sys
new_path = '/home/566/ad9701/drought_probability/'
if new_path not in sys.path:
    sys.path.append(new_path)

import xarray as xr
import numpy as np
import pandas as pd
import my_glmfit_funcs as my

main_dir = '/g/data/w97/ad9701/p_prob_analysis/temp_files/'
sub_dir = 'set5/'

varname = 'PminusEQ' # the name of the directory and file
vname = 'PminusEQ'   # the name of the variable inside the files
fname = varname + '_*_*_*.nc'

# select thresholds
threshList = [20, 50, 100]

# select timescales for analysis
ts = [2] #[2, 6, 12] #[2, 6, 8, 12]

# get the sst predictors
sst_dir = '/g/data/w97/ad9701/p_prob_analysis/sst_data/'
pNames = ['soi', 'sami', 'dmi', 'nino34_anom', 'nino4_anom']
pFiles = ['soi_monthly.nc', 'newsam.1957.2021.nc', 'dmi.had.long.data.nc', 'nino34.long.anom.data.nc', 'nino4.long.anom.data.nc']
for p in np.arange(len(pNames)):
    ds_temp = xr.open_dataset(sst_dir + pFiles[p])
    if (p > 0):
        ds_p[pNames[p]]=ds_temp[pNames[p]]
    else:
        ds_p = ds_temp
    del ds_temp

# select the predictors to include in the model
predSel = ['soi', 'dmi']
formula = 'response ~ +soi+dmi'
parameter = ['Intercept']
parameter.extend(predSel)
    
# create a new df of sample points at which 'predictions' will be made using the fitted model
time_slice = slice('1911-01-01','2020-05-31')
ds_p_subset = ds_p.sel(time = time_slice)
pred_dict = {}
for p in pNames:
    pred_dict.update({p: ds_p_subset[p].values})
pred_dict.update({"season": ds_p_subset['time.season'].values})    # add season to the sst predictors    
pred_df = pd.DataFrame(pred_dict, index = ds_p_subset['time'])     # make a dataframe of predictors
pred_df_sel = pred_df[predSel]
x_new = my.createSampleDf(pred_df_sel, list(pred_df_sel.keys()))

from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)
client

# from dask.distributed import Client,Scheduler
# from dask_jobqueue import SLURMCluster
# cluster = SLURMCluster(cores=4,memory="31GB")
# client = Client(cluster)
# cluster.scale(cores=4)
# client

for iW in ts:
    # get data
    data_dir = main_dir + varname + '_week' + str(iW) + '/' + sub_dir
    ds = xr.open_mfdataset(data_dir + fname, chunks = {'lat':400, 'lon':400})
    da_var = ds[vname].chunk(chunks = {'lat':40,'lon':40,'time':-1}).groupby('time.season')
    
    # select predictors for the same time points as the P-E or P-E-Q data
    da_time_bymon = np.array(pd.to_datetime(ds.time).to_period('M').to_timestamp().floor('D'))
    ds_p_sel = ds_p.sel(time = da_time_bymon)
    ds_p1_sel_gb = ds_p_sel[predSel[0]].groupby('time.season')
    ds_p2_sel_gb = ds_p_sel[predSel[1]].groupby('time.season')
    
    da_params_list = []
    da_pvalues_list = []
    da_prob_list = []
    da_aic_list = []
    seas_names = ['DJF', 'MAM', 'JJA', 'SON']
    dask_gufunc_kwargs = {'output_sizes':{"parameter": len(parameter), "sample":len(x_new)}}
    for seas in seas_names:    
        da_logistReg = xr.apply_ufunc(
            my.fit_logistReg_2Pred,                      # first the function, this function returns a tuple (GLM params, GLM pvalues, GLM modelled probabilities)
            da_var[seas],                                # function arg
            ds_p1_sel_gb[seas].values,
            ds_p2_sel_gb[seas].values,
            predSel,                                     #      "
            threshList,                                  #      "
            formula,                                     #      "
            x_new[predSel[0]].values,                    #      "
            x_new[predSel[1]].values,                    #      "
            input_core_dims=[["time"], ["time"], ["time"], ["predictors"], ["threshold"], [], ["sample"], ["sample"]],   # list with one entry per arg, these are the dimensions not to be broadcast
            output_core_dims=[["parameter", "threshold"], ["parameter", "threshold"], ["sample", "threshold"], ["threshold"]],                                # dimensions of the output
            vectorize=True,                                                                                                                    # broadcast over non-core dimensions of the input object?
            dask="parallelized",                                                                                                               # enable dask?
            dask_gufunc_kwargs=dask_gufunc_kwargs,                     
            output_dtypes=[float, float, float, float]
        )

        # assign co-ordinates add metadata
        new_coords_dict = {'threshold':threshList, 'parameter':parameter}
        da_params_list.append(da_logistReg[0].rename('glm_params').assign_coords(new_coords_dict))
        da_pvalues_list.append(da_logistReg[1].rename('glm_pvalues').assign_coords(new_coords_dict))
        da_aic_list.append(da_logistReg[3].rename('glm_aic').assign_coords({'threshold':threshList}))
        da_prob_temp = da_logistReg[2].rename('glm_probability').assign_coords({'threshold':threshList}).to_dataset()
        for k in list(x_new.keys()):
            da_prob_temp[k] = x_new[k]
        da_prob_temp = da_prob_temp.rename({'dim_0':'sample'})   
        da_prob_list.append(da_prob_temp)

    ds_all = xr.concat(da_prob_list, dim = 'season').assign_coords({'season':seas_names})
    ds_all['glm_params'] = xr.concat(da_params_list, dim = 'season').assign_coords({'season':seas_names})
    ds_all['glm_pvalues'] = xr.concat(da_pvalues_list, dim = 'season').assign_coords({'season':seas_names})
    ds_all['glm_aic'] = xr.concat(da_aic_list, dim = 'season').assign_coords({'season':seas_names})

    # save file
    out_file = data_dir + 'GLM_results_' + '_'.join(predSel) + '_bySeason_test.nc'
    ds_all.to_netcdf(out_file)
    
# cluster.scale(cores=0)
# client
# ! squeue -u ad9701

CPU times: user 1min 14s, sys: 16.9 s, total: 1min 31s
Wall time: 24min 9s


In [4]:
ds_all['glm_probability']

<xarray.DataArray 'glm_probability' (season: 4, lat: 141, lon: 261, sample: 9, threshold: 3)>
dask.array<concatenate, shape=(4, 141, 261, 9, 3), dtype=float64, chunksize=(1, 40, 40, 9, 3), chunktype=numpy.ndarray>
Coordinates:
  * lat        (lat) float32 -39.0 -38.95 -38.9 -38.85 ... -32.1 -32.05 -32.0
  * lon        (lon) float32 139.0 139.1 139.1 139.1 ... 151.9 151.9 151.9 152.0
  * threshold  (threshold) int64 20 50 100
  * sample     (sample) int64 0 1 2 3 4 5 6 7 8
  * season     (season) <U3 'DJF' 'MAM' 'JJA' 'SON'

In [7]:
import sys

new_path = '/home/566/ad9701/drought_probability/'
if new_path not in sys.path:
    sys.path.append(new_path)
    
import my_glmfit_funcs as glm
out_dir = '/g/data/w97/ad9701/p_prob_analysis/temp_files/'
varname = 'PminusEQ'
threshold = [20, 50, 100]
nSets = 13

for iWeek in [2]:
    sub_dir_names = []
    for i in range(2, 2+nSets):
        sub_dir_names.append('/set' + str(i))
    for i in [4]: # range(3, 4): # range(nSets):
        sub_dir = sub_dir_names[i]
        ds_all = glm.fit_gridded_logistReg(main_dir = out_dir, varname = varname, iWeek = iWeek, threshold = threshold, sub_dir = sub_dir)

Full set of sst predictors: ['soi', 'sami', 'dmi', 'nino34_anom', 'nino4_anom']
Selected sst predictors: ['soi', 'dmi']


In [ ]:
predSel = ['soi', 'dmi']
data_dir = out_dir + varname + '_week' + str(iWeek) + '/' + sub_dir + '/'
out_file = data_dir + 'GLM_results_' + '_'.join(predSel) + '_bySeason.nc'
ds_all.to_netcdf(out_file)

In [9]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 4
Total threads: 4,Total memory: 11.23 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:45087,Workers: 4
Dashboard: /proxy/8787/status,Total threads: 4
Started: 30 minutes ago,Total memory: 11.23 GiB
Comm: tcp://10.0.128.149:43149,Total threads: 1
Dashboard: /proxy/38553/status,Memory: 2.81 GiB
Nanny: tcp://127.0.0.1:42683,
